In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import *
from sklearn.feature_selection import *

from imblearn.under_sampling import RandomUnderSampler


In [122]:
length_index = 15 
shop_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/short_trajectory/shopper_data_slm_feat_length_{}.csv'.format(length_index))
shop_df.drop(['session_id_hash', 'product_action', 'reduced_time', 'HVGms_edges', 'pattern_hvg_4_nodes', 'pattern_hvg_5_nodes', 'unigram', 'bigram', 'trigram', 'unigram_prob_freq', 'bigram_prob_freq', 'trigram_prob_freq', 'pattern_hvg_4_nodes_prob_freq', 'pattern_hvg_5_nodes_prob_freq'], axis=1, inplace=True)
shop_df['conversion_class'] = np.where(shop_df['conversion_class'] == 'NC', 0, 1)
print(shop_df.shape)
shop_df.head()

(5674, 128)


,conversion_class,unigram_entropy,bigram_entropy,trigram_entropy,pattern_hvg_4_nodes_entropy,pattern_hvg_5_node_entropy,"(1,)","(2,)","(3,)","(4,)",...,Q5,V5,O5,R5,T5,S5,U5,C5,I5,H5
0,0,-0.000000,-0.000000,-0.000000,1.011404,1.329661,1.000000,0.000000,0.0,0.0,...,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.500402,1.028514,1.524707,1.329661,1.791759,0.800000,0.200000,0.0,0.0,...,0.166667,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.673012,1.004242,1.118969,1.039721,1.386294,0.600000,0.400000,0.0,0.0,...,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.579915,0.758937,0.937156,1.332179,1.332179,0.266667,0.733333,0.0,0.0,...,0.000000,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.500402,1.028514,1.410514,1.277034,1.549826,0.800000,0.200000,0.0,0.0,...,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
x, y = shop_df.drop('conversion_class', axis=1), shop_df['conversion_class']
rus = RandomUnderSampler(random_state=42)
subsamples = []
for _ in range(10):
    x_resampled, y_resampled = rus.fit_resample(x, y)
    subsamples.append((x_resampled, y_resampled))
    

In [124]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(4539, 127) (1135, 127) (4539,) (1135,)


# MI (Mutual Information)

In [125]:
mi_feat_list_10_percentile = []
mi_feat_list_20_percentile = []
mi_feat_list_30_percentile = []
mi_feat_list_50_percentile = []
mi_feat_list_75_percentile = []
mi_feat_list_90_percentile = []


for samples in subsamples:
    x_sample = samples[0]
    y_sample = samples[1]
    
    sel_10_percentile_cols = SelectPercentile(mutual_info_classif, percentile=10)
    sel_10_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_10_percentile.append(x_sample.columns[sel_10_percentile_cols.get_support()])
    
    sel_20_percentile_cols = SelectPercentile(mutual_info_classif, percentile=20)
    sel_20_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_20_percentile.append(x_sample.columns[sel_20_percentile_cols.get_support()])
    
    sel_30_percentile_cols = SelectPercentile(mutual_info_classif, percentile=30)
    sel_30_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_30_percentile.append(x_sample.columns[sel_30_percentile_cols.get_support()])
    
    sel_50_percentile_cols = SelectPercentile(mutual_info_classif, percentile=50)
    sel_50_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_50_percentile.append(x_sample.columns[sel_50_percentile_cols.get_support()])
    
    sel_75_percentile_cols = SelectPercentile(mutual_info_classif, percentile=75)
    sel_75_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_75_percentile.append(x_sample.columns[sel_75_percentile_cols.get_support()])
    
    sel_90_percentile_cols = SelectPercentile(mutual_info_classif, percentile=90)
    sel_90_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_90_percentile.append(x_sample.columns[sel_90_percentile_cols.get_support()])

print(len(mi_feat_list_10_percentile[0]))
print(len(mi_feat_list_20_percentile[0]))
print(len(mi_feat_list_30_percentile[0]))
print(len(mi_feat_list_50_percentile[0]))
print(len(mi_feat_list_75_percentile[0]))
print(len(mi_feat_list_90_percentile[0]))

print(len(mi_feat_list_10_percentile))
print(len(mi_feat_list_20_percentile))
print(len(mi_feat_list_30_percentile))
print(len(mi_feat_list_50_percentile))
print(len(mi_feat_list_75_percentile))
print(len(mi_feat_list_90_percentile))

13
26
38
63
95
114
10
10
10
10
10
10


# mRMR

In [126]:
def mrmr_feat_selected(x, y, f, correlation, k):
    feat_list = []
    selected = []
    not_selected = x.columns.to_list()
    # repeat k times
    for i in range(k):
    
        if i > 0:
            last_selected = selected[-1]
            correlation.loc[not_selected, last_selected] = x[not_selected].corrwith(x[last_selected]).abs().clip(.00001)
            
        score = f.loc[not_selected] / correlation.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)
        
    feat_list.append(selected)
    return feat_list

In [127]:
mrmr_feat_list_10_percentile = []
mrmr_feat_list_20_percentile = []
mrmr_feat_list_30_percentile = []
mrmr_feat_list_50_percentile = []
mrmr_feat_list_75_percentile = []
mrmr_feat_list_90_percentile = []

for samples in subsamples:
    x_sample = samples[0]
    y_sample = samples[1]
    
    F = pd.Series(f_regression(x_sample, y_sample)[0], index = x_sample.columns)
    corr = pd.DataFrame(.001, index = x_sample.columns, columns = x_sample.columns)

    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_10_percentile[0]))
    mrmr_feat_list_10_percentile += x
    
    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_20_percentile[0]))
    mrmr_feat_list_20_percentile += x
    
    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_30_percentile[0]))
    mrmr_feat_list_30_percentile += x
    
    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_50_percentile[0]))
    mrmr_feat_list_50_percentile += x
    
    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_75_percentile[0]))
    mrmr_feat_list_75_percentile += x

    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_90_percentile[0]))
    mrmr_feat_list_90_percentile += x
        
print(len(mrmr_feat_list_10_percentile[0]))
print(len(mrmr_feat_list_20_percentile[0]))
print(len(mrmr_feat_list_30_percentile[0]))
print(len(mrmr_feat_list_50_percentile[0]))
print(len(mrmr_feat_list_75_percentile[0]))
print(len(mrmr_feat_list_90_percentile[0]))

print(len(mrmr_feat_list_10_percentile))
print(len(mrmr_feat_list_20_percentile))
print(len(mrmr_feat_list_30_percentile))
print(len(mrmr_feat_list_50_percentile))
print(len(mrmr_feat_list_75_percentile))
print(len(mrmr_feat_list_90_percentile))

13
26
38
63
95
114
10
10
10
10
10
10


# MI & mRMR

In [128]:
mi_mrmr_feat_list_10_percentile = []
for feat in zip(mi_feat_list_10_percentile, mrmr_feat_list_10_percentile):
    mi_mrmr_feat_list_10_percentile.append(list(set(feat[0]).intersection(feat[1])))

mi_mrmr_feat_list_20_percentile = []
for feat in zip(mi_feat_list_20_percentile, mrmr_feat_list_20_percentile):
    mi_mrmr_feat_list_20_percentile.append(list(set(feat[0]).intersection(feat[1])))
    
mi_mrmr_feat_list_30_percentile = []
for feat in zip(mi_feat_list_30_percentile, mrmr_feat_list_30_percentile):
    mi_mrmr_feat_list_30_percentile.append(list(set(feat[0]).intersection(feat[1])))
    
mi_mrmr_feat_list_50_percentile = []
for feat in zip(mi_feat_list_50_percentile, mrmr_feat_list_50_percentile):
    mi_mrmr_feat_list_50_percentile.append(list(set(feat[0]).intersection(feat[1]))) 
    
mi_mrmr_feat_list_75_percentile = []
for feat in zip(mi_feat_list_75_percentile, mrmr_feat_list_75_percentile):
    mi_mrmr_feat_list_75_percentile.append(list(set(feat[0]).intersection(feat[1]))) 

mi_mrmr_feat_list_90_percentile = []
for feat in zip(mi_feat_list_90_percentile, mrmr_feat_list_90_percentile):
    mi_mrmr_feat_list_90_percentile.append(list(set(feat[0]).intersection(feat[1]))) 
    
print(len(mi_mrmr_feat_list_10_percentile[0]))
print(len(mi_mrmr_feat_list_20_percentile[0]))
print(len(mi_mrmr_feat_list_30_percentile[0]))
print(len(mi_mrmr_feat_list_50_percentile[0]))
print(len(mi_mrmr_feat_list_75_percentile[0]))
print(len(mi_mrmr_feat_list_90_percentile[0]))

10
20
26
44
78
107


# Saving in CSV and Text Files

In [129]:
i = 1
for samples in subsamples:
    sample_df = pd.concat([samples[0], samples[1]], axis = 1)
    sample_df.to_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/subsamples/{}/subsample_{}.csv'.format(length_index, i))
    i+=1

In [130]:
def write_list_of_lists_to_file(list_of_lists, file_path):
    with open(file_path, 'w') as file:
        for sublist in list_of_lists:
            line = ' '.join(str(element) for element in sublist) + '\n'
            file.write(line)

In [131]:
write_list_of_lists_to_file(mi_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_10_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_20_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_30_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_50_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_75_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_75_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_90_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_90_percentile.txt'.format(length_index))

write_list_of_lists_to_file(mrmr_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_10_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_20_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_30_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_50_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_75_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_75_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_90_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_90_percentile.txt'.format(length_index))

write_list_of_lists_to_file(mi_mrmr_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_10_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_20_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_30_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_50_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_75_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_75_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_90_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_90_percentile.txt'.format(length_index))

In [132]:

# https://towardsdatascience.com/mrmr-explained-exactly-how-you-wished-someone-explained-to-you-9cf4ed27458b